In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]


In [ ]:
sats_games = [os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/sats*')]
# sats_games = ['sats_3regions_3licenses_5types']
# sats_games = ['sats_2regions_2licenses_2players_5types']
# sats_games = ['rw_game_2_war_speedy_2']
sats_games = ['sats_3regions_3licenses_5types']
sats_games

In [ ]:
# SATS Games
CONFIG = 'aug11'
BR_CONFIG = 'aug11'
N_SEEDS = 5
overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 1000 --eval_zero true --eval_every_early 200 --total_timesteps 1_000_000 --dispatch_br true --br_overrides "--total_timesteps 10_000"'
for GAME_NAME in sats_games:
    dispatch.dispatch_experiments(CONFIG, base_job_name='speedy_war3', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_require_br', game_name=f'{GAME_NAME}', overrides = overrides + ' --iterate_br false --require_br true', n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_random_ic', game_name=f'{GAME_NAME}', overrides = overrides + ' --random_ic true', n_seeds=N_SEEDS)

In [ ]:
albert_games = [os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/rw*')]
# albert_games = ['rw_game_2_settle', 'rw_game_2_settle_spiteful', 'rw_game_2_settle_very_spiteful']
# albert_games = ['rw_game_2_settle_spiteful']
albert_games

In [ ]:
# Albert's games without iterate_br
CONFIG = 'may17'
BR_CONFIG = 'may17'
N_SEEDS = 1
overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 100_000 --eval_zero true --eval_every_early 20_000 --num_training_episodes 100_000 --br_overrides "--num_training_episodes 10_000"'
for GAME_NAME in albert_games:
    dispatch.dispatch_experiments(CONFIG, base_job_name='adam_test_3', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='albert_games_revived_3_with_require_br', game_name=f'{GAME_NAME}', overrides = overrides + ' --iterate_br false --require_br true', n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='albert_games_revived_3_with_random_ic', game_name=f'{GAME_NAME}', overrides = overrides + ' --random_ic true', n_seeds=N_SEEDS)

In [ ]:
# CONFIG = 'flatmlp'
# GAME_NAME = 'very_large_game_2'
# N_SEEDS = 1
# overrides = f'--overwrite_db True --br_portfolio_path flatmlp --eval_every 1_000_000 --eval_zero true --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
# dispatch.dispatch_experiments(CONFIG, base_job_name='april11_flattest_very_large_2', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

In [ ]:
CONFIG = 'mar17'
GAME_NAME = 'large_game_2'
N_SEEDS = 5
overrides = f'--br_portfolio_path mar17 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'

# Base
dispatch.dispatch_experiments(CONFIG, base_job_name='mar24', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

# Hidden demands  
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_hide', overrides = overrides, n_seeds=N_SEEDS)

# Reserve prices
# TODO: Holding off for now b/c rounds issue
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_low_reserve', overrides = overrides, n_seeds=N_SEEDS)
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_same_reserve', overrides = overrides, n_seeds=N_SEEDS)

# Activity
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_activity', overrides = overrides, n_seeds=N_SEEDS)
# Undersell
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_undersell', overrides = overrides, n_seeds=N_SEEDS)

# Initial pricing experiment, on the base game
dispatch.dispatch_experiments(CONFIG, base_job_name='mar24', game_name=f'{GAME_NAME}_pricing05', overrides = overrides, n_seeds=N_SEEDS)

In [ ]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "rnrtest" | awk '{print $1}' | xargs scancel

In [ ]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R"

In [ ]:
!squeue | wc -l

In [ ]:
!scancel -u ubuntu

In [ ]:
# !scancel  25809

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from auctions.webutils import *

def find_best_checkpoint(run, max_t=None):
    ev_df = parse_run(run, max_t)
    best_t = ev_df.groupby('t')['ApproxNashConv'].first().idxmin()
    nash_conv_by_t = ev_df.groupby('t')['ApproxNashConv'].first()
    best_checkpoint = get_checkpoint(run, t=best_t)
    return nash_conv_by_t, best_checkpoint, nash_conv_by_t.min()


run = EquilibriumSolverRun.objects.last()
find_best_checkpoint(run)

In [ ]:
Experiment.objects.get(name='large_game_2_no_activity.json').delete()

In [ ]:
Game.objects.get(name='large_game_2_pricing_extreme.json').delete()

In [ ]:
!python /apps/open_spiel/web/auctions/manage.py nfsp --seed 109 --filename rounds.json --network_config_file mar17/lstm --experiment_name lstmtest --job_name "lstm_for_greg" --dispatch_br false --br_portfolio_path feb22 --eval_every 10_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --report_freq 10_000 --overwrite_db true

In [ ]:
list(Experiment.objects.all().values_list('name', flat=True))

In [ ]:
r = EquilibriumSolverRun.objects.get(name='large_game_3b_hide-mar17lstm-100')

In [ ]:
from auctions.webutils import *
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r)

In [ ]:
list(BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name'))

In [ ]:
BestResponse.objects.filter(checkpoint=pk)[0].brevaluation.__dict__

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r).values('best_response__name')

In [ ]:
BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name')

In [ ]:
experiments = ['mar24', 'mar22_4']
all_scripts = []
for e in experiments:
    scripts = glob.glob(f'/shared/outputs/{e}/*/evaluations/*.sh')
    scripts = [s for s in scripts if 'straightforward' not in s]
    scripts = [s for s in scripts if s[-5:] == "_2.sh"]
    all_scripts += scripts

In [ ]:
for script in all_scripts:
    parent = Path(script).parent
    !cd {parent} && sbatch {script}

In [ ]:
x = Evaluation.objects.last()

In [ ]:
x.samples['allocations']

In [ ]:
Evaluation.objects.last()

In [ ]:
import torch.nn.functional as F
import torch
x = torch.tensor([1,2,3])

In [ ]:
F.pad(x, (0, 2))

In [ ]:
torch.nn.functional.pad(input, pad, mode='constant', value=0.0)

In [ ]:
python /apps/open_spiel/web/auctions/manage.py nfsp --seed 100 --filename sats_5.json --network_config_file flatmlp/mlp --experiment_name test_require_br --job_name "sats_5_test-flatmlpmlp-100" --dispatch_br false --dry_run true --overwrite_db True --br_portfolio_path flatmlp --eval_every 1_000_000 --eval_zero false --num_training_episodes 8_000_000 --br_overrides "--num_training_episodes 1_000_000" --iterate_br false --require_br true

# Get env_and_model


# run_nfsp(env_and_model, 10_000, True, False, NullResultSaver(), 1, False, NullDispatcher(), eval_every, eval_every_early, eval_exactly, eval_zero, report_freq, dispatch_br, agent_selector, random_ic=False):

In [ ]:
BREvaluation.objects.filter(best_response__name='straightforward').last().expected_value_cdf

In [ ]:
!pip install wandb

In [ ]:
Evaluation.objects.last().samples.keys()

In [3]:
Evaluation.objects.filter(
                    checkpoint__equilibrium_solver_run__experiment__name='testevolution3',
                    checkpoint__equilibrium_solver_run__generation=0,
                ).count()

0

In [2]:
Evaluation.objects.last()

<Evaluation: Evaluation for rw_game_2_war_speedy_2-aug11ppo-100 (speedy_war3) Iteration 62464>

NameError: name 'EnvParams' is not defined

In [6]:
!python /apps/cluster_open_spiel/web/auctions/manage.py ppo --filename sats_2regions_2licenses_2players_5types.json --network_config_file aug11/ppo_reward --experiment_name testrewardshape --job_name "parameters-aug11ppo-104" --overwrite_db True --eval_every 500 --eval_zero true --total_timesteps 1_000_000 --use_wandb true --dry_run true

/home/ubuntu/.pyenv/versions/venv/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/home/ubuntu/.pyenv/versions/venv/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/home/ubuntu/.pyenv/versions/venv/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
[2022-10-04 23:11:24,477] {ubc_utils.py:44} INFO - Setting numpy and torch seed to 1234
[2022-10-04 23:11:24,478] {ppo_utils.py:43} INFO - Reading config from /apps/open_spiel/notebooks/configs/aug11/ppo_reward.yml
[2022-10-04 23:11:24,480] {ubc_utils.